In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import fbeta_score,make_scorer

In [2]:
from simple_elmo import ElmoModel

model = ElmoModel()

In [3]:
model.load('193')

2021-03-13 20:30:35,916 : INFO : Loading model from 193...
2021-03-13 20:30:35,945 : INFO : We will cache the vocabulary of 100 tokens.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


2021-03-13 20:30:37,471 : WARNING : From c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\simple_elmo\model.py:529: LSTMCell.__init__ (from tensorflow.python.keras.layers.legacy_rnn.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


2021-03-13 20:30:37,535 : WARNING : From c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\simple_elmo\model.py:570: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Please use `layer.add_weight` method instead.


2021-03-13 20:30:37,734 : WARNING : From c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:962: Layer.add_variable (from tensorflow.python.keras.engine.base_layer_v1) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2021-03-13 20:30:38,091 : WARNING : From c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:970: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


'The model is now loaded.'

In [4]:
train = pd.read_json('snli_1.0_train.jsonl', lines = True)
test = pd.read_json('snli_1.0_test.jsonl', lines = True)
val = pd.read_json('snli_1.0_dev.jsonl', lines = True)
full = {'train': train, 'val':val, 'test': test}

In [5]:
def delete_unsure_labels(data):
    data = data.drop(index = data[data.gold_label == '-'].index)
    return data


for frame in full:
    full[frame] = delete_unsure_labels(full[frame])

In [6]:
train = full['train']
val = full['val']
test = full['test']

In [7]:
def get_dist(data):
    sent1 = model.get_elmo_vector_average(list(data['sentence1']))
    sent2 = model.get_elmo_vector_average(list(data['sentence2']))
    return sent1 - sent2

In [12]:
train_sent1 = model.get_elmo_vector_average(list(train['sentence1']))
train_sent2 = model.get_elmo_vector_average(list(train['sentence2']))
X_train = train_sent1 - train_sent2
X_train.shape

2021-03-13 14:13:46,165 : INFO : Warming up ELMo on 32 sentences...
2021-03-13 14:13:54,203 : INFO : Warming up finished.
2021-03-13 14:13:54,243 : INFO : Texts in the current batch: 32
2021-03-13 14:14:01,474 : INFO : Texts in the current batch: 32
2021-03-13 14:14:07,694 : INFO : Texts in the current batch: 32
2021-03-13 14:14:18,194 : INFO : Texts in the current batch: 32
2021-03-13 14:14:23,202 : INFO : Texts in the current batch: 32
2021-03-13 14:14:27,525 : INFO : Texts in the current batch: 32
2021-03-13 14:14:30,085 : INFO : Texts in the current batch: 32
2021-03-13 14:14:35,493 : INFO : Texts in the current batch: 32
2021-03-13 14:14:40,948 : INFO : Texts in the current batch: 32
2021-03-13 14:14:52,959 : INFO : Texts in the current batch: 32
2021-03-13 14:14:58,186 : INFO : Texts in the current batch: 32
2021-03-13 14:15:07,136 : INFO : Texts in the current batch: 32
2021-03-13 14:15:13,762 : INFO : Texts in the current batch: 32
2021-03-13 14:15:19,000 : INFO : Texts in the 

2021-03-13 14:26:29,061 : INFO : Texts in the current batch: 32
2021-03-13 14:26:36,410 : INFO : Texts in the current batch: 32
2021-03-13 14:26:44,464 : INFO : Texts in the current batch: 32
2021-03-13 14:26:51,395 : INFO : Texts in the current batch: 32
2021-03-13 14:26:56,454 : INFO : Texts in the current batch: 32
2021-03-13 14:27:04,312 : INFO : Texts in the current batch: 32
2021-03-13 14:27:10,402 : INFO : Texts in the current batch: 32
2021-03-13 14:27:14,939 : INFO : Texts in the current batch: 32
2021-03-13 14:27:20,260 : INFO : Texts in the current batch: 32
2021-03-13 14:27:26,820 : INFO : Texts in the current batch: 32
2021-03-13 14:27:35,776 : INFO : Texts in the current batch: 32
2021-03-13 14:27:42,384 : INFO : Texts in the current batch: 32
2021-03-13 14:27:48,167 : INFO : Texts in the current batch: 32
2021-03-13 14:27:53,063 : INFO : Texts in the current batch: 32
2021-03-13 14:27:59,146 : INFO : Texts in the current batch: 32
2021-03-13 14:28:07,722 : INFO : Texts i

2021-03-13 14:39:53,585 : INFO : Texts in the current batch: 32
2021-03-13 14:39:59,147 : INFO : Texts in the current batch: 32
2021-03-13 14:40:04,801 : INFO : Texts in the current batch: 32
2021-03-13 14:40:08,614 : INFO : Texts in the current batch: 32
2021-03-13 14:40:15,149 : INFO : Texts in the current batch: 32
2021-03-13 14:40:21,620 : INFO : Texts in the current batch: 32
2021-03-13 14:40:29,329 : INFO : Texts in the current batch: 32
2021-03-13 14:40:34,473 : INFO : Texts in the current batch: 32
2021-03-13 14:40:40,856 : INFO : Texts in the current batch: 32
2021-03-13 14:40:45,939 : INFO : Texts in the current batch: 32
2021-03-13 14:40:50,585 : INFO : Texts in the current batch: 32
2021-03-13 14:40:55,329 : INFO : Texts in the current batch: 32
2021-03-13 14:41:03,207 : INFO : Texts in the current batch: 32
2021-03-13 14:41:11,491 : INFO : Texts in the current batch: 32
2021-03-13 14:41:16,088 : INFO : Texts in the current batch: 32
2021-03-13 14:41:20,728 : INFO : Texts i

ResourceExhaustedError: OOM when allocating tensor with shape[32,94,45,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node bilm/CNN/Conv2D_5 (defined at c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\simple_elmo\model.py:380) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Errors may have originated from an input operation.
Input Source operations connected to node bilm/CNN/Conv2D_5:
 bilm/embedding_lookup/Identity_1 (defined at c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\simple_elmo\model.py:346)

Original stack trace for 'bilm/CNN/Conv2D_5':
  File "c:\users\julia\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\julia\appdata\local\programs\python\python38\lib\runpy.py", line 87, in

In [ ]:
#X_train = get_dist(train)
#y_train = train['gold_label']
X_val = get_dist(val)
y_val = val['gold_label']
X_test = get_dist(test)
y_test = test['gold_label']

2021-03-13 20:33:26,151 : INFO : Warming up ELMo on 32 sentences...
2021-03-13 20:33:41,144 : INFO : Warming up finished.
2021-03-13 20:33:41,305 : INFO : Texts in the current batch: 32
2021-03-13 20:33:54,262 : INFO : Texts in the current batch: 32
2021-03-13 20:34:02,700 : INFO : Texts in the current batch: 32
2021-03-13 20:34:09,886 : INFO : Texts in the current batch: 32
2021-03-13 20:34:17,172 : INFO : Texts in the current batch: 32
2021-03-13 20:34:26,196 : INFO : Texts in the current batch: 32
2021-03-13 20:34:31,401 : INFO : Texts in the current batch: 32
2021-03-13 20:34:44,516 : INFO : Texts in the current batch: 32
2021-03-13 20:34:50,227 : INFO : Texts in the current batch: 32
2021-03-13 20:35:00,425 : INFO : Texts in the current batch: 32
2021-03-13 20:35:08,053 : INFO : Texts in the current batch: 32
2021-03-13 20:35:16,946 : INFO : Texts in the current batch: 32
2021-03-13 20:35:26,993 : INFO : Texts in the current batch: 32
2021-03-13 20:35:34,667 : INFO : Texts in the 

2021-03-13 20:51:36,956 : INFO : Texts in the current batch: 32
2021-03-13 20:51:45,942 : INFO : Texts in the current batch: 32
2021-03-13 20:51:53,417 : INFO : Texts in the current batch: 32
2021-03-13 20:52:01,601 : INFO : Texts in the current batch: 32
2021-03-13 20:52:14,525 : INFO : Texts in the current batch: 32
2021-03-13 20:52:27,963 : INFO : Texts in the current batch: 32
2021-03-13 20:52:34,785 : INFO : Texts in the current batch: 32
2021-03-13 20:52:40,551 : INFO : Texts in the current batch: 32
2021-03-13 20:52:50,396 : INFO : Texts in the current batch: 32
2021-03-13 20:52:58,422 : INFO : Texts in the current batch: 32
2021-03-13 20:53:05,620 : INFO : Texts in the current batch: 32
2021-03-13 20:53:11,558 : INFO : Texts in the current batch: 32
2021-03-13 20:53:16,706 : INFO : Texts in the current batch: 32
2021-03-13 20:53:22,455 : INFO : Texts in the current batch: 32
2021-03-13 20:53:28,131 : INFO : Texts in the current batch: 32
2021-03-13 20:53:36,608 : INFO : Texts i

2021-03-13 21:08:03,170 : INFO : Texts in the current batch: 32
2021-03-13 21:08:10,896 : INFO : Texts in the current batch: 32
2021-03-13 21:08:21,740 : INFO : Texts in the current batch: 32
2021-03-13 21:08:30,093 : INFO : Texts in the current batch: 32
2021-03-13 21:08:36,646 : INFO : Texts in the current batch: 32
2021-03-13 21:08:44,081 : INFO : Texts in the current batch: 32
2021-03-13 21:08:51,324 : INFO : Texts in the current batch: 32
2021-03-13 21:08:57,654 : INFO : Texts in the current batch: 32
2021-03-13 21:09:05,642 : INFO : Texts in the current batch: 32
2021-03-13 21:09:12,873 : INFO : Texts in the current batch: 32
2021-03-13 21:09:34,938 : INFO : Texts in the current batch: 32
2021-03-13 21:09:42,255 : INFO : Texts in the current batch: 32
2021-03-13 21:09:51,823 : INFO : Texts in the current batch: 32
2021-03-13 21:10:01,315 : INFO : Texts in the current batch: 32
2021-03-13 21:10:08,928 : INFO : Texts in the current batch: 32
2021-03-13 21:10:16,525 : INFO : Texts i

2021-03-13 21:21:06,071 : INFO : Texts in the current batch: 32
2021-03-13 21:21:13,603 : INFO : Texts in the current batch: 32
2021-03-13 21:21:17,628 : INFO : Texts in the current batch: 32
2021-03-13 21:21:24,688 : INFO : Texts in the current batch: 32
2021-03-13 21:21:33,106 : INFO : Texts in the current batch: 32
2021-03-13 21:21:40,719 : INFO : Texts in the current batch: 32
2021-03-13 21:21:51,160 : INFO : Texts in the current batch: 32
2021-03-13 21:21:57,161 : INFO : Texts in the current batch: 32
2021-03-13 21:22:00,880 : INFO : Texts in the current batch: 32
2021-03-13 21:22:05,421 : INFO : Texts in the current batch: 32
2021-03-13 21:22:08,823 : INFO : Texts in the current batch: 32
2021-03-13 21:22:12,744 : INFO : Texts in the current batch: 32
2021-03-13 21:22:16,106 : INFO : Texts in the current batch: 32
2021-03-13 21:22:21,047 : INFO : Texts in the current batch: 32
2021-03-13 21:22:25,440 : INFO : Texts in the current batch: 32
2021-03-13 21:22:30,267 : INFO : Texts i

2021-03-13 21:32:16,264 : INFO : Texts in the current batch: 32
2021-03-13 21:32:19,635 : INFO : Texts in the current batch: 32
2021-03-13 21:32:23,780 : INFO : Texts in the current batch: 32
2021-03-13 21:32:28,247 : INFO : Texts in the current batch: 32
2021-03-13 21:32:33,041 : INFO : Texts in the current batch: 32
2021-03-13 21:32:37,814 : INFO : Texts in the current batch: 32
2021-03-13 21:32:44,988 : INFO : Texts in the current batch: 32
2021-03-13 21:32:51,719 : INFO : Texts in the current batch: 32
2021-03-13 21:33:03,826 : INFO : Texts in the current batch: 32
2021-03-13 21:33:09,595 : INFO : Texts in the current batch: 32
2021-03-13 21:33:13,122 : INFO : Texts in the current batch: 32
2021-03-13 21:33:17,457 : INFO : Texts in the current batch: 32
2021-03-13 21:33:23,715 : INFO : Texts in the current batch: 32
2021-03-13 21:33:27,816 : INFO : Texts in the current batch: 32
2021-03-13 21:33:33,193 : INFO : Texts in the current batch: 32
2021-03-13 21:33:38,229 : INFO : Texts i

2021-03-13 21:45:32,364 : INFO : Texts in the current batch: 32
2021-03-13 21:45:37,687 : INFO : Texts in the current batch: 32
2021-03-13 21:45:53,692 : INFO : Texts in the current batch: 32
2021-03-13 21:46:00,754 : INFO : Texts in the current batch: 32
2021-03-13 21:46:11,918 : INFO : Texts in the current batch: 32
2021-03-13 21:46:26,114 : INFO : Texts in the current batch: 32
2021-03-13 21:46:37,672 : INFO : Texts in the current batch: 32
2021-03-13 21:47:03,367 : INFO : Texts in the current batch: 32
2021-03-13 21:47:12,269 : INFO : Texts in the current batch: 32
2021-03-13 21:47:22,044 : INFO : Texts in the current batch: 32
2021-03-13 21:47:29,996 : INFO : Texts in the current batch: 32
2021-03-13 21:47:35,806 : INFO : Texts in the current batch: 32
2021-03-13 21:47:41,619 : INFO : Texts in the current batch: 32
2021-03-13 21:47:48,038 : INFO : Texts in the current batch: 32
2021-03-13 21:47:53,977 : INFO : Texts in the current batch: 32
2021-03-13 21:48:04,304 : INFO : Texts i

2021-03-13 22:06:16,255 : INFO : Texts in the current batch: 32
2021-03-13 22:06:24,813 : INFO : Texts in the current batch: 32
2021-03-13 22:06:33,046 : INFO : Texts in the current batch: 32
2021-03-13 22:06:41,116 : INFO : Texts in the current batch: 32
2021-03-13 22:06:52,584 : INFO : Texts in the current batch: 32
2021-03-13 22:07:02,817 : INFO : Texts in the current batch: 32
2021-03-13 22:07:16,651 : INFO : Texts in the current batch: 32
2021-03-13 22:07:28,707 : INFO : Texts in the current batch: 32
2021-03-13 22:07:46,538 : INFO : Texts in the current batch: 32
2021-03-13 22:07:57,717 : INFO : Texts in the current batch: 32
2021-03-13 22:08:10,646 : INFO : Texts in the current batch: 32
2021-03-13 22:08:23,707 : INFO : Texts in the current batch: 32
2021-03-13 22:08:30,734 : INFO : Texts in the current batch: 32
2021-03-13 22:08:49,921 : INFO : Texts in the current batch: 32
2021-03-13 22:09:00,275 : INFO : Texts in the current batch: 32
2021-03-13 22:09:10,643 : INFO : Texts i

In [ ]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
ftwo_scorer = make_scorer(fbeta_score, beta=3, average='weighted')
cross_val_score(clf, X_train, y_train, cv=3, scoring= ftwo_scorer)

In [ ]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
clf.fit(X_train, y_train)
y_train_preds = clf.predict(X_train)
f_train = fbeta_score(y_train, y_train_preds, beta=10, average=None,)
y_val_preds = clf.predict(X_val)
f_val = fbeta_score(y_val, y_val_preds, beta=10, average=None,)
print(f'train: {f_train}')
print(f'val: {f_val}')